In [166]:
from data.function import Stock,Categories
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime
import numpy as np


In [232]:
def convert_daily_to_monthly(data,period):
    # Use the resample function to get the open price on the first day of the month
    monthly_open = data['open'].resample(period).first()

    # Use the resample function to get the close price on the last day of the month
    monthly_close = data['close'].resample(period).last()

    # Use the resample function to get the maximum and minimum values for each month
    monthly_high = data['high'].resample(period).max()
    monthly_low = data['low'].resample(period).min()

    # Use the resample function to get the total volume for each month
    monthly_volume = data['volume'].resample(period).sum()
    # Create a new DataFrame with the monthly stock prices
    monthly_prices = pd.DataFrame({'open': monthly_open,
                                    'high': monthly_high,
                                    'low': monthly_low,
                                    'close': monthly_close,                                
                                    'volume': monthly_volume})

    monthly_prices = monthly_prices.reset_index()
    return monthly_prices

def get_close_date(data):
    close_day = [data['x'][i].strftime('%Y-%m-%d') for i in range(len(data['x'])) if np.isnan(data['open'][i])]
    # close_day = []
    # for i in range(len(data['x'])):
    #     if np.isnan(data['open'][i]):
    #         close_day.append(data['x'][i].strftime('%Y-%m-%d'))
    return close_day

  #____________________________________________________________________
    # fig = go.Figure(go.Candlestick(**data.drop(columns='volume')))# Create fig object
# def candle_plot(data,close_day):

#     # figure with secondary y-axis
#     fig = make_subplots(specs=[[{"secondary_y": True}]])

#     # candlestick
#     fig.add_trace(
#     go.Candlestick(**data.drop(columns='volume'),
#         showlegend=False),
#     row=1,
#     col=1,
#     secondary_y=True
# )


#     # volume
#     fig.add_trace(
#         go.Bar(x=data.x,
#             y=data.volume,
#             showlegend=False,
#             marker={
#                 "color": "#CCCCCC",
#             }),
#         secondary_y=False,
#     )

  
    
#     if len(data['x']) <= 1:# If data have 1 data or less than
#         pass
    
#     elif str(data['x'][1] - data['x'][0]) == '1 days 00:00:00':# If interval is 1 day
#         print('day')
#         fig.update_xaxes(
#         rangebreaks=[
#             dict(values=close_day)
#             ]
#         )
    
#     elif str(data['x'][1]- data['x'][0]) == '1:00:00':# If interval is 1 hour
#         print('hour')
#         fig.update_xaxes(
#         rangebreaks=[
#             dict(bounds=[17, 10], pattern="hour")
#             ]
#         )


#     fig.show()


                   
def candle_plot(data,close_day):

    # figure with secondary y-axis
    fig = make_subplots(rows=2,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=('OHLC', 'Volume'),
                    row_width=[0.3, 0.7])
    # candlestick
    fig.add_trace(
        go.Candlestick(**data.drop(columns='volume'),
            showlegend=False),
        row=1,
        col=1
    )

    # volume
    fig.add_trace(
        go.Bar(x=data.x,
            y=data.volume,
            showlegend=False),
        row=2,
        col=1
    )
    if len(data['x']) <= 1:# If data have 1 data or less than
        pass
    
    elif str(data['x'][1] - data['x'][0]) == '1 days 00:00:00':# If interval is 1 day
        print('day')
        fig.update_xaxes(
        rangebreaks=[
            dict(values=close_day)
            ]
        )
    
    elif str(data['x'][1]- data['x'][0]) == '1:00:00':# If interval is 1 hour
        print('hour')
        fig.update_xaxes(
        rangebreaks=[
            dict(bounds=[17, 10], pattern="hour")
            ]
        )
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

In [233]:
start_date = '2020-12-30'
end_date = '2022-12-31'

# Get stock price
PTT = Stock("PTT")
stock_price = PTT.get_all_stock_price()

# convert data to pandas dataframe
df = pd.DataFrame(stock_price)
df = df.rename(columns={0: 'x', 1: 'open', 2: 'high', 3: 'low', 4: 'close',5:'volume'})
df['x'] = pd.to_datetime(df['x'])
df.set_index('x', inplace=True)# Set the Date column as the index of the DataFrame

monthly_data = convert_daily_to_monthly(df,"D")# Convert daily data to monyhly or quarterly

close_day = get_close_date(monthly_data)# Find Close date

candle_plot(monthly_data,close_day)

day
